# get aging

This is a text cell. Start editing!

In [1]:
val runtime = Runtime.getRuntime
val maxMemory = runtime.maxMemory / 1024 / 1024  // Convert bytes to MB
println(s"Max memory: $maxMemory MB")

Max memory: 36864 MB


In [2]:
import org.apache.spark.{io => spark_io}
import org.apache.spark.sql.types._
import scala.reflect.runtime.universe._
import org.apache.spark.storage.StorageLevel
import org.apache.spark.rdd._
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import group.research.aging.spark.extensions._
import group.research.aging.spark.extensions.functions._
import kernel.display.html

In [3]:
import better.files._
import java.io.{File => JFile}

In [4]:
val data = File("/data")
val s2orc = data / "papers" / "s2orc" 
val all_pubmed_file = s2orc / "all_pubmed.parquet"
val all_aging_pubmed_file = s2orc / "all_aging_pubmed.parquet"

In [15]:
val all_pubmed = spark.read.parquet(all_pubmed_file.pathAsString)
all_pubmed.show(10,100)

+---------+--------------------+-----------------------------+----------------------------------------------------------------------------------------------------+----------------------------+----------------------------------------+----------------------+---------------+-----------------+-----------------------------+-----------------------------+---------------+------------------+-------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------

Aging paper

In [20]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.col

def filterContent(df: DataFrame, terms: List[String]): DataFrame = {
  val regex = terms.map(term => s"\\b$term\\b").mkString("(", "|", ")")
  df.filter(col("content_text").rlike("(?i)" + regex))  // Added (?i) for case-insensitive matching
}

In [21]:
val terms_aging = List("aging", "ageing", "lifespan extension", "healthspan", "longevity")
val all_aging = filterContent(all_pubmed, terms_aging)
all_aging.writeParquet(all_aging_pubmed_file.pathAsString)

Test

In [24]:
val all_aging_df = spark.read.parquet(all_aging_pubmed_file.pathAsString)
all_aging_df.count()

Moskalevs papers

In [16]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.array_contains

def filterByAuthor(df: DataFrame, author: String): DataFrame = {
  df.filter(array_contains(df("annotations_author"), author))
}